# Introduction to Keras #

## https://keras.io/ ## 
(Keras has great documentation, in fact, most of the examples below are taken in part of in full from the docs!)

## Keras workflow for a sequential model -- MNIST toy example ##

In [ ]:
import keras 


A **Sequential** model class is a linear stack of layers. 
Each layer has it's own class. Some examples that you might encounter frequently are included in the import statements below 

In [ ]:
#import the Sequential model class. 
from keras.models import Sequential 

#Core layers 
from keras.layers.core import Dense, Activation,Dropout,Flatten

#Convolution layers 
from keras.layers.convolutional import Conv1D, Conv2D 

#Pooling layers 
from keras.layers.pooling import MaxPooling1D, MaxPooling2D, AveragePooling1D, AveragePooling2D 

#Recurrent layers 
from keras.layers.recurrent import Recurrent, SimpleRNN, GRU, LSTM

#Embedding layers 
from keras.layers.embeddings import Embedding

#Merge layers 
from keras.layers.merge import Add, Multiply, Average, Maximum, Concatenate, Dot

#Normalization layers 
from keras.layers.normalization import BatchNormalization 


In [ ]:
#the MNIST dataset is included with keras installation 
from keras.datasets import mnist 
(X_train,y_train),(X_test,y_test)=mnist.load_data() 

In [ ]:
#we want to create a validation set to illustrate training with a validation dataset, so we "hack" this dataset by 
#splitting the test data into a test and validation dataset 
X_valid=X_test[0:int(X_test.shape[0]/2)]
y_valid=y_test[0:int(y_test.shape[0]/2)]
X_test=X_test[int(X_test.shape[0]/2)::]
y_test=y_test[int(y_test.shape[0]/2)::]

In [ ]:
#Let's briefly examine our data  
print("Training X:"+str(X_train.shape))
print("Training y:"+str(y_train.shape))
print("Valid X:"+str(X_valid.shape))
print("Valid y:"+str(y_valid.shape))
print("Test X:"+str(X_test.shape))
print("Test y:"+str(y_test.shape))

In [ ]:
%matplotlib inline
from matplotlib.pyplot import imshow
#You can visualize the digits that we classify 
digit_index=0
imshow(X_train[digit_index])
print("training label:"+str(y_train[digit_index]))

In [ ]:
#plot our first test digit -- let's see if we can train a deep learning model to correctly predict this digit. 
imshow(X_test[digit_index])


In [ ]:
#pre-process the input data 
from keras import backend as K 
from keras.utils import np_utils 
# input image dimensions                                                                                                                                                
img_rows, img_cols = 28, 28
#number of output classes
nb_classes=10 


#WARNING! order of dimensions differs for theano & tensorflow (that's why we check the backend and re-arrange the image 
#dimensions accordingly)
if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    X_valid = X_valid.reshape(X_valid.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    X_valid = X_valid.reshape(X_valid.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

    
#this is done to normalize the data, as RGB values range in intensity from 0 to 255 
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_valid = X_valid.astype('float32')
X_train /= 255
X_test /= 255
X_valid /= 255 

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_valid.shape[0], 'test samples')


# convert class vectors to binary class matrices                                                                                                                        
y_train = np_utils.to_categorical(y_train, nb_classes)
y_valid = np_utils.to_categorical(y_valid, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

print("y_train shape:", y_train.shape)


In [ ]:
K.image_dim_ordering()

### We want to implement the following model architecture to train a CNN to recognize handwritten digits: 
````    
 Convolution2D
     |
     v
   ReLU
     |
     v
Convolution2D
     |
     v
   ReLU
     |
     v
 MaxPool2D
     |
     v
  Dropout 
     |
     v
  Flatten
     |
     v
   Dense
     |
     v
    ReLU
     |
     v
  Dropout
     |
     v
   Dense
     |
     v
  Softmax
````

In [ ]:
#architecture hyperparameters 
# number of convolutional filters to use                                                                                                                                
nb_filters = 32
# size of pooling area for max pooling                                                                                                                                  
pool_size = (2, 2)
# convolution kernel size                                                                                                                                               
kernel_size = (3, 3)

In [ ]:
#model architecture 
model = Sequential()

model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]),
                        padding='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

#### A quick note on "padding" parameter: 

* With padding="valid" you get an output that is smaller than the input because the convolution is only computed where the input and the filter fully overlap.


* With padding="same" you get an output that is the "same" size as the input. That means that the filter has to go outside the bounds of the input by "filter size / 2" - the area outside of the input is normally padded with zeros.


* Note that some libraries also support the padding="full" where the filter goes even further outside the bounds of the input - up to "filter size - 1". This results in an output shape larger than the input.



In [ ]:
#compile the model 

#we can try several different optimizers 
#optimizer="adam"
#optimizer="adadelta"
#optimizer="adagrad"
optimizer="sgd"

#add momentum, alter learning rate: 
from keras.optimizers import SGD
#optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


In [ ]:
#from keras.utils import visualize_util
#visualize_util.plot(model,to_file="model.png")

In [ ]:
#training
batch_size=128
nb_epoch=10
history=model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,verbose=1, validation_data=(X_valid, y_valid),shuffle=True)


In [ ]:
# summarize history for accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()


In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
#training with fit generator 
import random 

#note: in a real "use case" x_matrix & y_matrix would likely be stored in an hdf5 file rather than loaded into memory. 
#check out http://www.h5py.org/
def create_generator(x_matrix,y_matrix,samples_to_yield): 
    num_entries=x_matrix.shape[0]
    upper_bound=num_entries-samples_to_yield
    while 1: 
        batch_index=random.randint(0,upper_bound)
        x=x_matrix[batch_index:batch_index+samples_to_yield]
        y=y_matrix[batch_index:batch_index+samples_to_yield]
        yield x,y


#create the generator for training data 
train_generator=create_generator(X_train,y_train,batch_size)
#create the generator for validation data 
valid_generator=create_generator(X_valid,y_valid,batch_size)

samples_per_epoch=3000 
nb_val_samples=1000

nb_epoch=2 #for illustrative purposes, we'll use 2 epochs to save time 
history=model.fit_generator(train_generator, samples_per_epoch, nb_epoch, verbose=1, 
                    callbacks=[], validation_data=valid_generator, validation_steps=nb_val_samples)


In [ ]:
#evaluate 
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


#evaluate with generator (for large dataset)
evaluation_generator=create_generator(X_test,y_test,batch_size)
nb_eval_samples=X_test.shape[0] 
score=model.evaluate_generator(evaluation_generator,nb_eval_samples)
print("With evaluate_generator:")
print('Test score:', score[0])
print('Test accuracy:', score[1])


In [ ]:
#predict classes
class_predictions=model.predict_classes(X_test) 

#predict probabilities 
class_probabilities=model.predict_proba(X_test)


In [ ]:
#let's look at the prediction for our test digit of interest 
print("predicted class:"+ str(class_predictions[digit_index]))
print("predicted probability:"+str([round(i,2) for i in class_probabilities[digit_index]]))

## Keras functional API ##
The Keras functional API is the way to go for defining complex models, such as multi-output models, directed acyclic graphs, or models with shared layers.

Use the functional API if you have: 

* multiple inputs & outputs 
* bypass layers
* merge layers 
* basically any non-linear connection between layers 


There are a few key points to remember for the functional API: 

* A layer instance is callable (on a tensor), and it returns a tensor
* Input tensor(s) and output tensor(s) can then be used to define a Model
* Such a model can be trained just like Keras Sequential models.
* All models are callable (just like layers). 

For example, let's look at a simple model that includes all layers required in the computation of output **b** given input **a**.

In [ ]:
#The simplest possible example for the functional API 
from keras.models import Model
from keras.layers import Input, Dense

a = Input(shape=(32,))
b = Dense(32)(a)
model = Model(inputs=a, outputs=b)


Useful attributes of Model

* `model.layers` is a flattened list of the layers comprising the model graph.
* `model.inputs` is the list of input tensors.
* `model.outputs` is the list of output tensors.

It's very straightforward to extend this model formulation to multi-input and multi-output models: 



In [ ]:
#example: multiple inputs and multiple outputs with the functional API 
a1 = Input(shape=(32,))
a2 = Input(shape=(32,))

b1 = Dense(32)(a1)
b2 = Dense(32)(a2)

model = Model(inputs=[a1, a2], outputs=[b1, b2])


In [ ]:
#let's examine the attributes of the model 
model.layers

In [ ]:
model.inputs

In [ ]:
model.outputs

## Example: Word embedding & training an LSTM with the functional API ## 
 
The functional API makes it easy to manipulate a large number of intertwined datastreams.

Let's consider the following model. We seek to predict how many retweets and likes a news headline will receive on Twitter. The main input to the model will be the headline itself, as a sequence of words, but to spice things up, our model will also have an auxiliary input, receiving extra data such as the time of day when the headline was posted, etc. The model will also be supervised via two loss functions. 

Here's what our model looks like:
![title](images/multi-input-multi-output-graph.png)

The main input will receive the headline, as a sequence of integers (each integer encodes a word). The integers will be between 1 and 10,000 (a vocabulary of 10,000 words) and the sequences will be 100 words long.

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense,Merge
from keras.layers.merge import Concatenate
from keras.models import Model

# headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# note that we can name any layer by passing it a "name" argument.
main_input = Input(shape=(100,), dtype='int32', name='main_input')

# this embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# a LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
lstm_out = LSTM(32)(x)

Here we insert the auxiliary loss, allowing the LSTM and Embedding layer to be trained smoothly even though the main loss will be much higher in the model.

In [ ]:
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

At this point, we feed into the model our auxiliary input data by concatenating it with the LSTM output:

In [ ]:
auxiliary_input = Input(shape=(5,), name='aux_input')
x = Concatenate()([lstm_out, auxiliary_input])

# we stack a deep fully-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# and finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

This defines a model with two inputs and two outputs:

In [ ]:
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

We compile the model and assign a weight of 0.2 to the auxiliary loss. To specify different loss_weights or loss for each different output, you can use a list or a dictionary. Here we pass a single loss as the loss argument, so the same loss will be used on all outputs.

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              loss_weights=[1., 0.2])

We can train the model by passing it lists of input arrays and target arrays:

In [ ]:
#model.fit([headline_data, additional_data], [labels, labels],
#          nb_epoch=50, batch_size=32)

Since our inputs and outputs are named (we passed them a "name" argument), We could also have compiled the model via:

In [ ]:
model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

# and trained it via:
#model.fit({'main_input': headline_data, 'aux_input': additional_data},
#          {'main_output': labels, 'aux_output': labels},
#          nb_epoch=50, batch_size=32)

## Switching the keras backend (theano vs tensorflow) ## 

You can switch the backend by editing the file **~/.keras/keras.json**

In [ ]:
! cat ~/.keras/keras.json

To use the tensorflow backend, just change "theano"  to "tensorflow". And you're done! 

## Getting fancy -- writing your own keras layers ##

Here is the skeleton of a Keras layer. There are only three methods you need to implement:

* **build(input_shape)**: this is where you will define your weights. Trainable weights should be added to the list self.trainable_weights. Other attributes of note are: self.non_trainable_weights (list) and self.updates (list of update tuples (tensor, new_tensor)). For an example of how to use non_trainable_weights and updates, see the code for the BatchNormalization layer.

* **call(x)**: this is where the layer's logic lives. Unless you want your layer to support masking, you only have to care about the first argument passed to call: the input tensor.

* **get_output_shape_for(input_shape)**: in case your layer modifies the shape of its input, you should specify here the shape transformation logic. This allows Keras to do automatic shape inference.


In [ ]:
#skeleton code for a keras layer
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np

class MyLayer(Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dim = input_shape[1]
        initial_weight_value = np.random.random((input_dim, output_dim))
        self.W = K.variable(initial_weight_value)
        self.trainable_weights = [self.W]

    def call(self, x, mask=None):
        return K.dot(x, self.W)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], self.output_dim)